# Neuroevolución

El Machine Learning, en términos simples, corresponde a un área de la Ciencia de Datos dedicada al desarrollo de arquitecturas y algoritmos para la extracción de descriptores o bien, `features`, a partir de generalmente un gran volumen de datos. Por supuesto, el tipo de datos que se utilice como también el propósito o problema para el cual estos son procesados, determinará el tipo de algoritmo a ser implementado.

Entre los algoritmos más comunes dentro del Machine Learning, se encuantran las ya bastante conocidas Rede Neuronales o Neural Networks (NN) que consisten en arquitecturas bioinspiradas en la forma en que nuestros cerebros procesan información mediante la transmisión paralela y secuencial de señales eléctricas, entre las neuronas. Así, las Redes Neuronales consisten en una red de nodos, generalmente ordenados por capas, que mediante una serie de ponderaciones y conexiones convierten una serie de valores de entrada (`input`) en una serie de valores de salida (`output`).

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_05/bin/neural_diagram.png" height="220">


Cada Nodo o Neurona se compone a su vez de una serie de `Inputs` $X_i$ que son ponderados numéricamente por los pesos o `Weights` $W_i$ para computar finalmente el `Output` $Y$ de la neurona mediante la función de activación $F(x)$. Además, dentro de la función de activación se introduce el valor `bias` $b$ como umbral de activación.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_05/bin/neural_node.png" height="220">

$Y = F(W_1 \cdot X_1 + W_2 \cdot X_2 + W_3 \cdot X_3 + b) = F(W^T \cdot X + b)$

Como se puede ver en la ecuación, la operación que realiza una neurona no es más que una ponderación lineal sobre los valores de entrada, para luego pasar este resultado a su función de activación. Las funciones de activación más comunmente utilizadas son la `ReLU`, la `Sigmoide`, la `Tanh` y, por supuesto, la activación `Lineal`.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_05/bin/activation_functions.png" height="220">

Ahora, por muy simple que pueda parecer su estructura, dependiendo de como sean implementadas, las Redes Neuronales cuentan con la flexivilidad de poder realizar diferentes clases de tareas en una gran variedad de áreas e industrias. Desde dominar el Go y el ajedrez, hasta detectar la presencia de tumores en radiografías, las Redes Neuronales han demostrado ser una poderosa, y aún emergente, herramienta.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_05/bin/lucas_deniro.png" height="220">

En este workshop nos concentraremos en el campo del aprendizaje no supervizado y en particular, en el Reinforcement Learning. En el Reinforcement Learning, las Redes Neuronales son entrenadas (ajustan sus parámetros) para resolver una tarea específica mediante prueba y error, de la misma manera en que uno entrena un animal con recompensas ante aciertos y castigos ante equivocaciones. Una implementación directa de esta idea es la Neuroevolución donde los algoritmos evolutivos son combinados con Redes Neuronales para evolucionar una arquitectura de Red Neuronal capaz de cumplir con la tarea objetivo.

En este caso, con la finalidad de introducir el funcionamiento de la librería `neat` y la neuroevolución, se trabajará sobre un juego sencillo que llamaremos `Shrek Runner`. En este juego, `Shrek` debe saltar sobre las tuberías `Pipes` las cuales irán apareciendo a distintas alturas y, a medida que el juego avance, a mayor velocidad. En este sentido, el juego es bastante similar al `T-Rex Runner!` de Google. De este modo, mediante `neat`, desarrollaremos una red neuornal que monitorerá continuamente la posición de `Shrek` y las `Pipes` para así controlar el salto de `Shrek` y, eventualmente, lograr dominar el juego.

## Shrek Runner

El juego `Shrek Runner!` que utilizaremos en esta oportunidad se encuentra implementado en el github del curso roboticafcfm en el modulo `shrek`. De este modo, para importar sus clases y funcionalidades debemos cargar el github al entorno de Colab.

In [ ]:
# cargar repositorio roboticafcfm desde github
! git clone https://github.com/cherrerab/roboticafcfm.git
%cd /content/roboticafcfm/

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_05/bin/shrek_diagram.png" height="320">

Para familiarizarnos con el funcionamiento del juego, a continuación se ha implementado una demo bajo el nombre de `shrek_demo()`. Esta demo contiene todos los bloques necesarios para ejecutar el juego, desde la inicialización de los objetos principales (`Shrek`, `Pipe` y `Base`) hasta el `main loop` que finalmente se encarga de compilar cada frame del juego. No obstante, a modo de demostración `Shrek` está programado para saltar cada `12 frames` y cada vez que `Shrek` colisiona con algún `Pipe`, este es restaurado a una posición segura.

Adicionalmente esta función `shrek_demo()` permite registrar o grabar el juego a un archivo `.mp4`. Para esto debe definir previamente un nombre de archivo `FILE_NAME` y habilitar la variable `RECORD`. Como ejemplo, ejecute el siguiente bloque de código.

In [ ]:
from shrek import Shrek
from shrek import Pipe
from shrek import Base
from shrek import draw_frame, BKGR_IMG

import cv2

# tamaño del canvas
WIN_WIDTH = 576
WIN_HEIGHT = 512

# ------------------------------------------------------------------------------
# Shrek Runner! DEMO
def shrek_demo():
  global RECORD
  global FILE_NAME

  # ---
  # INICIALIZAR OBJETOS
  # inicializar shrek
  shrek = Shrek(112, 400)

  # inicializar pipes
  pipes = [Pipe(WIN_WIDTH)]

  # inicializar base
  base = Base(400)

  # ---
  # RECORD VIDEO
  if RECORD:
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    file_path = '/content/' + FILE_NAME
    out = cv2.VideoWriter(file_path, fourcc, 29.0, (WIN_WIDTH, WIN_HEIGHT))

  # ----------------------------------------------------------------------------
  # MAIN LOOP
  # inicializar frame_count
  frame_count = 0

  run = True
  # como demo se ejecutará el juego por 20 segs
  while run and frame_count<29*20:

    # ---
    # MOVER SHREK
    # mover shrek
    shrek.move()

    # saltos de prueba para demo (esto no debería estar)
    if frame_count%15 == 0:
      shrek.jump()

    # ---
    # MOVER PIPES
    # mover pipes
    for pipe in pipes:
      pipe.move()

      # ---
      # CHECK COLLISIONS
      # si shrek colisiona con pipe es game over
      if pipe.collide(shrek):
        # **corrección de posición como demo (esto no debería estar)**
        shrek.y = pipe.y

        # game over
        # run = False


      # agregar pipes para que el juego continue
      if pipe.x < shrek.x and not(pipe.passed):
        pipe.passed = True
        pipes.append(Pipe(WIN_WIDTH))

      # remover pipes que ya salieron del juego
      if (pipe.x + pipe.width) < 0:
        pipes.remove(pipe)
    
    # mover base
    base.move()

    # ---
    # RECORD VIDEO
    if RECORD:
      # dibujar frame de la ventana de juego
      canvas = draw_frame([shrek], pipes, base, frame_count)

      frame = canvas[:,:,:3]
      out.write(frame)

      if cv2.waitKey(1) & 0xFF == 27:
        break

    # incrementar contador de frames
    frame_count += 1

  # finalizar video
  if RECORD:
    out.release()

# ------------------------------------------------------------------------------
# ejecutar demo del juego
FILE_NAME = 'TEST_SHREK.mp4'
RECORD = True
shrek_demo()

Podrá notar que en la carpeta `/content/` se ha creado el archivo `TEST_SHREK.mp4`. Este archivo puede ser descargado y visualizado en su computador como cualquier archivo de video.

Como podrá notar del video, saltar cada `12 frames` no es la mejor estrategia para dominar `Shrek Runner!`. Ahora es cuando incorporamos la Neuroevolución para entrenar o bien, construir, una Red Neuronal que aprenda a controlar perfectamente el salto de `Shrek` durante el juego. En particular, ocuparemos NEAT (Neuroevolving of Aumenting Topologies) que consiste en una ingeniosa codificación de Redes Neuronales, desarrollada por Kenneth O. Stanley, que le permite ser compatible con los Algoritmos Evolutivos vistos en las clases anteriores. De este modo, mediante NEAT evolucionaremos una población de Redes Neuronales cuyo fitness estará dado por qué tan bien se desempeñan en el juego.

Para ser más claros, cada un de los genomas creados por NEAT se corresponderá con una Red Neuronal que, a partir de la información del juego (`shrek.y`, `pipe.x`, `pipe.y` como `input`), retornará como `output` un valor entre -1 y 1. En este caso si el `output` mayor que 0, se interpretará como que su `shrek` asociado debe saltar. Además, como cualquier algoritmo genético, para evaluar cada uno de los genomas es necesario definir una función de fitness `eval_genomes`. En este caso, el fitness de un `shrek` está dado por su `score` en el juego el cual es directamente proporcional a la cantidad de `frames` que ha sobrevivido en la ejecución del juego.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_05/bin/shrek_diagram.png" height="300"> <img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_05/bin/neat_diagram.png" height="300">

Tomando como plantilla la función `shrek_demo()`, modificaremos esta para crear una función `eval_genomes` que sea compatible con NEAT. No obstante, antes también debemos instalar la librería especializada de NEAT de python, llamada `neat-python`. Como siempre puede revisar su documentación en el siguiente link:
- https://neat-python.readthedocs.io/en/latest/

In [ ]:
!pip install neat-python

Ahora, para compatibilizar la librería `neat-python` con la ejecución de `Shrek Runner!` debemos tener presente que NEAT, como cualquier algoritmo evolutivo genera una población de `genomes`. Estos `genomes`, como se muestra en la imagen anterior, representan distintas redes neuronales, las cuales deben ser compiladas con el método `neat.nn.FeedForwardNetwork.create(genome, config)`. Luego, cada una de estas redes `neat.nn.FeedForwardNetwork` controlará un único `Shrek` durante la ejecución del juego.

Además, para reducir el tiempo de entrenamiento, evaluaremos paralelamente todas las redes neuronales en una misma ejecución, en vez de simular cada una por separado.

In [ ]:
def eval_genomes(genomes, config):
  """
  Ejecuta una simulación del juego asignando a cada red contenida en genomes un
  Shrek con el fin de obtener su fitness.

  El fitness de cada Shrek dependerá del score que obtenga durante la simulación.

  """
  global RECORD
  global FILE_NAME

  global GEN
  VEL = 10.0
  GEN += 1
  print('training {:03d} generation'.format(GEN), end='')

  # ---
  # INITIALIZE SHREKS
  # comience por crear tres listas que contengan los genomas, las redes
  # asociadas a los genomas, y los Shreks correspondientes
  # shreks, nets, genms
  
  
  # por cada uno de los genomas
  for genome_id, genome in genomes:
    # inicializar su fitness a cero, genome.fitness
    

    # compilar su red neuronal correspondiente
    # neat.nn.FeedForwardNetwork.create
    net = 

    # inicializar su shrek asociado, Shrek(112, 400)


    # agregar estos elementos a sus correpondientes listas
    


  # ---
  # INITIALIZE PIPES
  pipes = [Pipe(WIN_WIDTH, VEL)]

  # ---
  # INITIALIZE BASE
  base = Base(400, VEL)

  # ---
  # RECORD VIDEO
  if RECORD:
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    file_path = '/content/' + FILE_NAME
    out = cv2.VideoWriter(file_path, fourcc, 29.0, (WIN_WIDTH, WIN_HEIGHT))

  # inicializar frame_count y score
  frame_count = 0
  score = 0

  run = True
  # correr mientras exista un Shrek que no haya perdido o no se supere
  # el minuto de juego
  while run and frame_count < (29*60):

    # BREAK CONDITION
    # si no quedan Shreks en juego, finalizar
    

    # ---
    # INPUTS
    # obtener inputs a entregar a las redes neuronales

    # determinar cual es el Pipe de interés
    pipe_idx = 0
    if len(pipes) > 1 and shreks[0].x > pipes[0].x + pipes[0].width:
      pipe_idx = 1

    # MOVE SHREKS
    # mover Shrek (shrek.move()), incrementar su fitness y mediante el output de
    # su red neuronal determinar si debe saltar (shrek.jump())
    # recuerde que los Shreks están contenidos en la lista shreks
    for x, shrek in enumerate(shreks):
      # mover shrek

      # incrementar fitness de genms[x]

      # obtener INPUTS para la red net[x]
      INPUT_0 = 
      INPUT_1 = 
      INPUT_2 = 

      # las redes neat.nn.FeedForwardNetwork poseen el método .activate()
      OUTPUT = nets[x].activate((INPUT_0, INPUT_1, INPUT_2))

      # si el output es mayor que cero, saltar
      if OUTPUT[0] > 0.0:
        

    # mover Pipes
    for pipe in pipes:
      pipe.move()

      # CHECKEAR COLISIONES
      # si algún Shrek colisiona con pipe es game over para ese Shrek
      # debe removerlo de la lista de Shreks, así como también su red y genoma
      # de las listas correspondientes.
      for shrek in shreks:
        if pipe.collide(shrek):
          shrek_idx = shreks.index(shrek)

          # disminuir fitness
          genms[shrek_idx].fitness

          # remover elementos de las listas
          nets.pop( )
          genms.pop( )
          shreks.pop( )

      # agregar pipes para que el juego continue
      if pipe.x < shrek.x and not(pipe.passed):
        pipe.passed = True
        pipes.append(Pipe(WIN_WIDTH, VEL))

      # eliminar pipes que ya salieron de la ventana de juego
      if (pipe.x + pipe.width) < 0:
        pipes.remove(pipe)
        score += 1
    
    # RECORD VIDEO
    if RECORD:
      # dibujar frame de la ventana de juego
      canvas = draw_frame(shreks, pipes, base, frame_count)

      frame = canvas[:,:,:3]
      out.write(frame)

      if cv2.waitKey(1) & 0xFF == 27:
        break

    # mover base
    base.move()

    # incrementar contador de frames
    frame_count += 1

  print(', score: {:03d}'.format(frame_count))

  # finalizar video
  if RECORD:
    out.release()

Ahora podemos ejecutar el siguiente y último bloque de código. Para configurar los parámetros principales de NEAT se entrega el archivo /content/roboticafcfm/shrek/config-feedforward.txt. No obstante, sientase libre de modificarlo y subir uno personalizado, solo tenga cuidado de modificar la ubicación de config_path.

In [ ]:
import neat

# ------------------------------------------------------------------------------
def run_neat(config_file):
    """
    -> None

    Ejecuta el algoritmo de NEAT para entrenar una red neuronal que domine
    el juego de Shrek.

    :param config_file: path del archivo de configuración config-feedforward.txt

    :return: None
    """
    global RECORD
    global FILE_NAME
    FILE_NAME = 'TRAINED_SHREK.mp4'

    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # inicializar la población de genomas
    p = neat.Population(config)

    # entrenar por 50 generaciones
    global GEN; GEN = 0
    RECORD = False
    winner = p.run(eval_genomes, 50)

    # registrar generación en video
    RECORD = True
    p.run(eval_genomes, 1)

# ------------------------------------------------------------------------------
# definir ubicación del archivo de configuración
config_path = '/content/roboticafcfm/shrek/config-feedforward.txt'

# ejecutar NEAT
run_neat(config_path)